<a href="https://colab.research.google.com/github/Giffy/Mobility_AIBCN/blob/master/2_Telefonia_Cat_DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Install libraries (if required)

In [1]:
# Instalar PyMongo
! python -m pip install pymongo

    100% |████████████████████████████████| 409kB 11.8MB/s 


##  Connect to Google Drive

In [2]:
# Check is Link to Drive is OK
google = !if [ -d 'GDrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  from google.colab import drive
  drive.mount('/content/GDrive/')
!if [ -d 'GDrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/GDrive/
Connection to Google drive successful


##  Load data

In [0]:
# Download Open Data from 'Dades obertes catalunya': https://analisi.transparenciacatalunya.cat/browse?q=cobertura%20M%C3%B2bil
!wget "https://analisi.transparenciacatalunya.cat/api/views/g9ma-vbt8/rows.csv?accessType=DOWNLOAD"

!mv rows* coverageCatalunya.csv
!tar -czvf coverageCatalunya.tar.gz coverageCatalunya.csv

In [0]:
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --db catalunya --collection mobile --type csv --file coverageCatalunya.csv --drop --headerline

##  Connect to MongoDB database

In [0]:
import pymongo
from pymongo import MongoClient
# uri (uniform resource identifier) defines the connection parameters 
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [0]:
db = client.catalunya                           ##  Seleccionamos la base de datos con nombre "movil"
db.list_collection_names()                      ##  Listamos las colecciones disponibles
collection = db.mobile

##  Database Visualization

In [8]:
print (list (collection.find().limit(1)[1]))
print ('Total documents: ' + str(  collection.count_documents({'_id' : {'$ne': None }}) ))

['_id', 'timestamp_', 'Date', 'Hora_', 'Lat', 'Long_', 'Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'satellites', 'precision1', 'provider', 'Activitat', 'downloadSpeed', 'uploadSpeed', 'MUNICIPI', 'NOM_MUNI']
Total documents: 11744914


In [9]:
attributes = ['Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'satellites', 'precision1', 'provider', 'Activitat', 'downloadSpeed', 'uploadSpeed', 'MUNICIPI', 'NOM_MUNI']
attributes = ['Senyal', 'Xarxa', 'Operador', 'status', 'Desc_', 'net_type', 'speed', 'precision1', 'provider', 'Activitat', 'NOM_MUNI','Carrier']

for attribute in attributes:
  if (attribute != '_id'):
    print ( attribute + " "+ str( collection.distinct( attribute )))

Senyal [9, 25, 12, 14, 8, 2, 23, 16, 15, 3, 4, 5, 6, 11, 17, 21, 13, 7, 22, 33, 10, 19, 24, 28, 0, 27, 18, 30, 31, 29, 26, 20, 52, 1, 50, 60, 58, 32, 39, 35, 54, 51, 53, 56, 63, 42, 49, 55, 61, 59, 57, 62, 64, 37, 65, 40, 47, 36, 34, 45, 41, 38, 99, 43, 48, 44, 46, 98, 96, 78, 93, 87, 88, 97, 76, 86]
Xarxa ['MetroPCS', 'movistar', 'Telefonica Moviles Espana', 'orange', 'vodafone', 'yoigo', 'o2 - de', 'TICAE', 'F SFR', '', 'ENTEL', 'Orange SP', 'Solo llamadas de emergencia', 'VIVO', 'PARLEM', 'TIM', 'MOBILAND', 'France Telcom Espana SA', 'Sense servei.', 'Sin servicio', 'F-Bouygues Telecom', 'Orange F', 'T-Mobile', 'TELCEL', 'Lowi', "Nom├®s trucades d'emerg├¿ncia", 'vodafone UK', 'Lebara', 'BEL PROXIMUS', 'France Telecom Espa├▒a SA', 'PLAY', 'ION MOBILE', 'PLAY (T-Mobile)', 'P TMN', 'vodafone NL', 'L TANGO', 'Vodafone', 'vodafone P', 'A1', 'vodafone HU', 'Tele2', 'UNEFON', 'Claro', 'null', 'Solo emergencia', 'Voz Premium', 'vodafone IT', 'I WIND', 'Plus', 'MASMovil', 'CLARO BR', 'E-Plus

## Data modification


In [34]:
list(collection.find().limit(1))

[{'Activitat': 'IN_VEHICLE',
  'Date': '25/11/2015 08:20:18 PM',
  'Desc_': 'STATE_EMERGENCY_ONLY',
  'Hora_': '20:20:18',
  'Lat': 26.09727,
  'Long_': -81.73866,
  'MUNICIPI': '',
  'NOM_MUNI': '',
  'Operador': 'MetroPCS',
  'Senyal': 9,
  'Xarxa': 'MetroPCS',
  '_id': ObjectId('5c0bf2f3fade320f82a69967'),
  'downloadSpeed': '',
  'net_type': '2G',
  'precision1': 18,
  'provider': 'gps',
  'satellites': 0,
  'speed': 6.8,
  'status': 2,
  'timestamp_': 1448479218665,
  'uploadSpeed': ''}]

### Define Area of study

1- Lat: 41.453045    Long: 2.037423

2- Lat: 41.453045    Long: 2.256154
<br><br>

3- Lat: 41.338663    Long: 2.037423

4- Lat: 41.338663    Long: 2.256154

In [35]:
delete = collection.delete_many({'Long_': {"$gt" : 2.256154 }}) 
delete.deleted_count

2782972

In [36]:
delete = collection.delete_many({'Long_': {"$lt" : 2.037423 }}) 
delete.deleted_count

4619737

In [37]:
delete = collection.delete_many({'Lat': {"$gt" : 41.453045 }}) 
delete.deleted_count

1395474

In [38]:
delete = collection.delete_many({'Lat': {"$lt" : 41.338663 }}) 
delete.deleted_count

146115

### Deletion of no relevant fields :  downloadSpeed, uploadSpeed, satellites, MUNICIPI, etc...

In [39]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'downloadSpeed': 1, 'uploadSpeed': 1, 'satellites': 1, 'MUNICIPI': 1, 'speed': 1,
                     'status': 1, 'Senyal':1 , 'Desc_':1, 'provider': 1, 'timestamp_': 1, 'net_type': 1,
                     'NOM_MUNI': 1, 'Operador': 1, 'Xarxa': 1, 'precision1': 1}}
 
collection.update_many( filters ,fields)

In [63]:
list( collection.find().limit(1) )

[{'Activitat': 'ON_FOOT',
  'Date': '25/11/2015 08:20:35 PM',
  'Day': 25,
  'Hora_': '20:20:35',
  'Hour': 20,
  'Lat': 41.39286,
  'Latitude': 41.393,
  'Long_': 2.18633,
  'Longitude': 2.1865,
  'Month': 11,
  'Weekday': 3,
  'Year': 2015,
  '_id': ObjectId('5c0bf2f3fade320f82a6996b')}]

### Add Year, Month, Day, Weekday, Hour  fields


In [64]:
from bson.objectid import ObjectId
import datetime
import sys

def round_coord(x, base=0.0005):
     return round(base*round(x/base), 4)

consulta_tmp = (collection.find({},{'_id','Date','Hora_','Lat','Long_'}))
count = 0
num_docs = collection.count_documents({})
for item in consulta_tmp:
    pk = str(item['_id'])
    datee = str(item['Date'])
    day = int(datee.split('/')[0])
    month = int(datee.split('/')[1])
    year = int(datee.split('/')[2].split(' ')[0])
    horaa = str(item['Hora_'])
    hora = int(horaa.split(':')[0])
    weekday = datetime.datetime(year, month, day).isoweekday()
    lat = float(item['Lat'])
    long = float(item['Long_'])
    lat = round_coord(lat)
    long = round_coord(long)    
    filters = { '_id' : ObjectId(pk) }   
    fields = {'$set': {'Year': year, 'Month' : month ,'Day' : day , 'Weekday' : weekday , 'Hour' : hora, 'Latitude' : lat , 'Longitude' : long }}
    collection.update_one( filters ,fields)
    count += 1 
    if (count%50):    
      b = round((count*100 / num_docs),1)
      sys.stdout.write('\r'+ str(b) + "%")

100.0%

In [67]:
list(collection.find().limit(1))

[{'Activitat': 'ON_FOOT',
  'Day': 25,
  'Hora_': '20:20:35',
  'Hour': 20,
  'Latitude': 41.393,
  'Longitude': 2.1865,
  'Month': 11,
  'Weekday': 3,
  'Year': 2015,
  '_id': ObjectId('5c0bf2f3fade320f82a6996b')}]

In [66]:
filters = {'_id' : {'$ne': None }}
fields = {'$unset': {'Date' : 1, 'Lat': 1, 'Long_': 1}}

collection.update_many( filters ,fields)

## Export to CSV

In [70]:
# Export to CSV
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoexport --db catalunya --collection mobile --type csv --out coverageCat.csv --fields Activitat,Day,Hora_,Hour,Latitude,Longitude,Month,Weekday,Year,_id

2018-12-08T18:10:38.179+0000	connected to: localhost
2018-12-08T18:11:43.095+0000	exported 2800616 records


In [71]:
!tar -czvf coverageCat.tar.gz coverageCat.csv
!mv coverageCat.tar.gz GDrive/My\ Drive/datasets

coverageCat.csv
